This notebook continuously estimates the prevailing, realtime, positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>
<br>

## Preliminaries: In Progress

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.investigations.prospects
import atlantic.investigations.hospitalizations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

### Attributes

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 
         'hospitalizedIncrease': np.float64, 'positiveCumulative': np.float64, 'testCumulative': np.float64, 
         'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 'positiveRate': np.float64, 
         'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>

### Source URL

In [14]:
uribaseline = os.path.join(parent, 'warehouse', 'baselines.csv')
urihospitalizations = os.path.join(parent, 'warehouse', 'hospitalizations.csv')

<br>

### Data

In [15]:
baselines = pd.read_csv(filepath_or_buffer=uribaseline, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)
baselines.loc[:, 'deathPositiveRate'] = np.where(baselines['positiveRate'] > 0, 100 * baselines['deathRate'] / baselines['positiveRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15184 entries, 0 to 15183
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          15184 non-null  datetime64[ns]
 1   STUSPS                  15184 non-null  object        
 2   deathIncrease           15184 non-null  float64       
 3   deathCumulative         15184 non-null  float64       
 4   positiveIncrease        15184 non-null  float64       
 5   positiveCumulative      15184 non-null  float64       
 6   hospitalizedIncrease    15184 non-null  float64       
 7   hospitalizedCumulative  15184 non-null  float64       
 8   testIncrease            15184 non-null  float64       
 9   testCumulative          15184 non-null  float64       
 10  deathRate               15184 non-null  float64       
 11  positiveRate            15184 non-null  float64       
 12  testRate                15184 non-null  float6

In [16]:
hospitalizations = pd.read_csv(filepath_or_buffer=urihospitalizations, header=0, 
                               usecols=fields,dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

hospitalizations.loc[:, 'hospitalizedPositiveRate'] =  np.where(hospitalizations['positiveRate'] > 0, 
                                                                100 * hospitalizations['hospitalizedRate'] / hospitalizations['positiveRate'], 0)
hospitalizations.loc[:, 'deathHospitalizedRate'] =  np.where(hospitalizations['hospitalizedRate'] > 0, 
                                                             100 * hospitalizations['deathRate'] / hospitalizations['hospitalizedRate'], 0)

logger.info(hospitalizations.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10512 entries, 0 to 10511
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            10512 non-null  datetime64[ns]
 1   STUSPS                    10512 non-null  object        
 2   deathIncrease             10512 non-null  float64       
 3   deathCumulative           10512 non-null  float64       
 4   positiveIncrease          10512 non-null  float64       
 5   positiveCumulative        10512 non-null  float64       
 6   hospitalizedIncrease      10512 non-null  float64       
 7   hospitalizedCumulative    10512 non-null  float64       
 8   testIncrease              10512 non-null  float64       
 9   testCumulative            10512 non-null  float64       
 10  deathRate                 10512 non-null  float64       
 11  positiveRate              10512 non-null  float64       
 12  testRate          

INFO:__main__:None


<br>
<br>

## Regression

### $\small{Set-up}$

#### Latest Date

In [17]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2020-11-08 00:00:00


<br>
<br>

#### Periods

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [18]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

#### Partitions

In [19]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### **positives vs. tests**

#### Analysis

In [20]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [21]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

Estimates

In [22]:
logger.info(estimates.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   datetimeobject           52 non-null     datetime64[ns]
 1   gradient                 52 non-null     object        
 2   lowerconfidenceinterval  52 non-null     object        
 3   upperconfidenceinterval  52 non-null     object        
 4   pvalue                   52 non-null     object        
 5   rsquared                 52 non-null     object        
 6   intercept                52 non-null     object        
 7   STUSPS                   52 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 3.4+ KB


<br>

Predictions

In [23]:
logger.info(predictions.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   testRate  260 non-null    float64
 1   minimum   260 non-null    float64
 2   median    260 non-null    float64
 3   maximum   260 non-null    float64
 4   STUSPS    260 non-null    object 
dtypes: float64(4), object(1)
memory usage: 10.3+ KB


<br>

#### State of affairs

In [24]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [25]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [26]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
32,2020-11-08,0.0164091,0.0141961,0.0204742,1.25884e-19,0.994045,1387.24,NY,1.64091,3.377225,48.0,60.9417
8,2020-11-08,0.019251,0.0162546,0.0215555,1.74574e-19,0.991324,1030.36,DC,1.9251,3.240802,49.0,51.9453
45,2020-11-08,0.0323184,0.0219237,0.0487078,7.26224e-14,0.968659,-627.842,VT,3.23184,1.230567,50.0,30.9422
1,2020-11-08,0.0216152,0.00478852,0.0348593,9.68208e-07,0.795683,350.388,AK,2.16152,2.533907,51.0,46.2637
19,2020-11-08,0.0132908,0.00891642,0.024747,1.01366e-11,0.936966,-116.856,ME,1.32908,1.137489,52.0,75.2398


<br>
<br>

### **deaths vs. positives**

In [27]:
del formula, regressor, prospects, estimates, predictions

<br>

#### Analysis

In [28]:
formula = 'deathRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=20, num=5, latestdate=latestdate)

In [29]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

#### State of Affairs

In [30]:
dpr = estimates

<br>

Prevailing Death Positive Rate

In [31]:
dpr.loc[:, 'prevailingDPR'] = 100 * dpr['gradient']
dpr = dpr.merge(baselines[['datetimeobject', 'STUSPS', 'deathPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
dpr.loc[:, 'cf'] = dpr[['prevailingDPR', 'deathPositiveRate']].max(axis=1)

dpr.loc[:, 'rank'] = dpr['cf'].rank(method='min', ascending=False)
dpr.sort_values(by='rank', ascending=True, inplace=True)
dpr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [32]:
dpr.loc[:, 'prevailingTPC'] = np.where(dpr['prevailingDPR'] > 0, dpr['prevailingDPR'].rdiv(100), 0)
dpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'deathPositiveRate': 'continuousDPR'},
          inplace=True)
dpr.to_csv(path_or_buf=os.path.join(warehouse, 'dpr.csv'), index=False, header=True, encoding='utf-8')

dpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDPR,continuousDPR,rank,prevailingTPC
27,2020-11-08,0.00569154,0.00359293,0.00691683,1.37708e-13,0.967664,12.4231,NE,0.569154,0.853207,48.0,175.699
49,2020-11-08,0.00791918,0.00673756,0.00849554,1.92499e-19,0.994418,3.14165,WI,0.791918,0.848498,49.0,126.276
50,2020-11-08,0.00664704,0.00383959,0.0103728,9.15474e-12,0.949031,-0.318767,WY,0.664704,0.658579,50.0,150.443
44,2020-11-08,0.00309509,0.00264485,0.00365078,2.31102e-17,0.990365,7.82387,UT,0.309509,0.496905,51.0,323.092
1,2020-11-08,0.00287368,-0.000102802,0.00423496,1.77368e-07,0.826256,4.25875,AK,0.287368,0.423686,52.0,347.985


<br>
<br>

### **hospitalizations vs. positives**

In [33]:
del formula, regressor, prospects, estimates, predictions

<br>

#### Analysis

In [34]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [35]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

#### State of Affairs

In [36]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [37]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [38]:
hpr = hpr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
31,2020-11-08,0.0188573,0.0117236,0.0230952,2.88341e-15,0.981362,32.9096,WY,1.88573,32.0,2.917389,53.0298
32,2020-11-08,0.0129935,0.00371555,0.017687,1.63554e-11,0.945208,162.408,MA,1.29935,33.0,7.802263,76.9617
33,2020-11-08,0.0122068,0.00789877,0.0141496,4.01443e-14,0.975994,47.2894,NH,1.22068,34.0,6.358104,81.9218
34,2020-11-08,9.99201e-16,-0.00160302,0.026124,0.103446,0.110116,343.787,CT,9.99201e-14,35.0,15.688960,1.0008e+15
35,2020-11-08,2.22045e-16,-4.39487e-14,4.48368e-14,0.904063,-292.7,462.615,NY,2.22045e-14,35.0,17.011130,4.5036e+15


<br>
<br>

## Clean-up

In [39]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states